In [1]:
pip install pandas scikit-learn requests

Note: you may need to restart the kernel to use updated packages.


In [16]:
import requests

API_KEY = "997aa789a933aab2a5d72f41b5b928eb"
CITY = "port blair"

def get_weather_data(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "visibility": data["visibility"] / 1000,  # Convert to km
            "weather_condition": data["weather"][0]["main"]
        }
        return weather
    else:
        return None

weather_data = get_weather_data(CITY)
print(weather_data)

{'temperature': 29.02, 'humidity': 79, 'visibility': 6.0, 'weather_condition': 'Clouds'}


In [3]:
import pandas as pd

data = {
    "temperature": [15, 20, 10, 5, 12, 25, 30, 21.21],
    "humidity": [85, 60, 90, 80, 70, 40, 30, 80],
    "visibility": [2, 10, 1, 3, 8, 12, 15, 4],  # in km
    "weather_condition": ["Fog", "Clear", "Fog", "Rain", "Clear", "Clear", "Clear", "Mist"],
    "brightness_level": [80, 30, 90, 70, 50, 20, 10, 85]  # Target variable
}

df = pd.DataFrame(data)
print(df)

   temperature  humidity  visibility weather_condition  brightness_level
0        15.00        85           2               Fog                80
1        20.00        60          10             Clear                30
2        10.00        90           1               Fog                90
3         5.00        80           3              Rain                70
4        12.00        70           8             Clear                50
5        25.00        40          12             Clear                20
6        30.00        30          15             Clear                10
7        21.21        80           4              Mist                85


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['weather_condition'] = label_encoder.fit_transform(df['weather_condition'])

X = df.drop('brightness_level', axis=1)
y = df['brightness_level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 397.6625


In [20]:
def predict_brightness(model, weather_data):

    input_data = pd.DataFrame([weather_data])
    input_data['weather_condition'] = label_encoder.transform(input_data['weather_condition'])
    
    brightness = model.predict(input_data)[0]
    return brightness

live_weather = {
    "temperature": 29.02,
    "humidity": 79,
    "visibility": 6,
    "weather_condition": "Clear"
}
brightness_level = predict_brightness(model, live_weather)
print(f"Recommended Brightness Level: {brightness_level}")

Recommended Brightness Level: 59.15
